In [16]:
import pandas as pd

df = pd.read_csv("./swissvotes-dataset.csv", delimiter=";", na_values=".")
# Exclude data from votes with missing values (very first and of the last)
df = df.iloc[1:-12, :]
# Extract some data rows for comparison and verification
volk = df["volk"].iloc[:]
stand = df["stand"].iloc[:]
kt_ja = df["kt-ja"].iloc[:]
kt_nein = df["kt-nein"].iloc[:]
ktjaproz = df["ktjaproz"].iloc[:]

staende = kt_ja + kt_nein
invalid_results = (staende > 23).sum() + (staende < 22).sum()
print("Number of invalid staende results: {}".format(invalid_results))

Number of invalid staende results: 0


In [17]:
volk_wrong = (df["volk"] != (df["volkja-proz"] > 50.0)).sum()
print("Check calculation of volk: {}".format(volk_wrong))
delta_ktjaproz = (df["ktjaproz"] - (100.0 * df["kt-ja"] / staende)).abs().sum()
print("Total absolute sum of deltas for ktjaproz: {}".format(delta_ktjaproz))

delta_volkja_proz = (df["volkja-proz"] - (100.0 * df["volkja"] / df["gultig"])).abs().sum()
print("Total absolute sum of deltas for volkja-proz: {}".format(delta_volkja_proz))

delta_bet = (df["bet"] - (100.0 * df["stimmen"] / df["berecht"])).abs().sum()
print("Total absolute sum of deltas for bet: {}".format(delta_bet))

invalid_berecht = (df["berecht"] < df["stimmen"]).sum()
print("Number of invalid berecht: {}".format(invalid_berecht))

delta_stimmen = (df["stimmen"] - df["leer"] - df["ungultig"] - df["gultig"])
print("Total absolute sum of deltas for stimmen: {}".format(delta_stimmen.abs().sum()))
stimmen_discrepancies = (delta_stimmen != 0).sum()
print("Number of discrepancies in stimmen: {}".format(stimmen_discrepancies))

delta_gultig = (df["gultig"] - df["volkja"] - df["volknein"])
print("Total absolute sum of deltas for gultig: {}".format(delta_gultig.abs().sum()))
gultig_discrepancies = (delta_gultig != 0).sum()
print("Number of discrepancies in gultig: {}".format(gultig_discrepancies))

Check calculation of volk: 0
Total absolute sum of deltas for ktjaproz: 1.0515810276676572
Total absolute sum of deltas for volkja-proz: 1.5598724290052153
Total absolute sum of deltas for bet: 1.53731533820525
Number of invalid berecht: 0
Total absolute sum of deltas for stimmen: 55086.0
Number of discrepancies in stimmen: 58
Total absolute sum of deltas for gultig: 1638038.0
Number of discrepancies in gultig: 28


In [18]:
#df = df.reset_index(drop=True)
c_berecht = df.iloc[:, df.columns.str.endswith("-berecht")]
c_stimmen = df.iloc[:, df.columns.str.endswith("-stimmen")]
c_stimmen.columns = c_stimmen.columns.str.replace("-stimmen", "")
c_bet = df.iloc[:, df.columns.str.endswith("-bet")]
c_gultig = df.iloc[:, df.columns.str.endswith("-gultig")]
c_gultig.columns = c_gultig.columns.str.replace("-gultig", "")
c_ja = df.iloc[:, df.columns.str.endswith("-ja")]
c_ja = c_ja.drop(columns=["inserate-ja", "kt-ja"])
c_ja.columns = c_ja.columns.str.replace("-ja", "")
c_nein = df.iloc[:, df.columns.str.endswith("-nein")]
c_nein = c_nein.drop(columns=["inserate-nein", "kt-nein"])
c_nein.columns = c_nein.columns.str.replace("-nein", "")
c_japroz = df.iloc[:, df.columns.str.endswith("-japroz")]
c_japroz.columns = c_japroz.columns.str.replace("-japroz", "")
c_annahme = df.iloc[:, df.columns.str.endswith("-annahme")]
c_annahme.columns = c_annahme.columns.str.replace("-annahme", "")

c_discrepancies_gultig = (c_gultig - c_ja - c_nein != 0)
print("Discrepancies gultig at cantonal level: {}".format(c_discrepancies_gultig.abs().sum().sum()))
number_cantons = c_annahme.count(axis=1)
c_weight = c_annahme.count(axis=0)
c_weight[:] = 1.0
c_weight['ar','ai','ow','nw','bl','bs'] = 0.5

Discrepancies gultig at cantonal level: 900


In [25]:
wrong_result = (c_annahme != (c_japroz >= 50.0))
import numpy as np
coord = np.where(wrong_result)
coordinates = [(x,y) for x, y in zip(coord[0], coord[1])]

discrepancies_staende = ((c_annahme * c_weight).sum(axis=1) != kt_ja)
discrepancies_staende.index[discrepancies_staende == True]
stand_accepted = (c_annahme * c_weight).sum(axis=1) > (number_cantons/2)
(stand_accepted != stand).sum()

198

In [20]:
(number_cantons - staende - 3.0).sum()

0.0